In [1]:
import findspark

In [2]:
findspark.init("/home/danial/spark-3.3.2-bin-hadoop3")

In [3]:
from pyspark.sql import SparkSession

In [4]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [6]:
spark = SparkSession.builder.appName('NetworkWordCounter').getOrCreate()

In [7]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words 
# two built-in SQL functions - split and explode, to split each line into multiple rows with a word each. 
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

23/04/17 15:21:52 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [ ]:
# nc -lk 9999

In [ ]:
 # Start running the query that prints the running counts to the console
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

23/04/17 15:22:41 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-661d74b4-d754-4407-9e96-fd1f6e67b494. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/04/17 15:22:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    2|
+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------+-----+
|    word|count|
+--------+-----+
|   words|    1|
|   hello|    2|
|    with|    1|
|      is|    1|
|       a|    1|
|    this|    1|
|    line|    1|
|multiple|    1|
+--------+-----+



In [ ]:
# Spark streaming - Twitter Project 